In [4]:
#dataset=OCTID, improved customCNN with CLAHE, advanced augmentation, mixup, label smoothing, and sharp Cosine LR scheduler
# Final CustomCNN++ for OCTID on Local Machine (Optimized)
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, LearningRateScheduler

# Paths
base_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project"
dataset_path = os.path.join(base_path, "OCTID/Train")
logs_path = os.path.join(base_path, "logs")
training_path = os.path.join(logs_path, "training")
result_path = os.path.join(logs_path, "results/customCNN_OCTID_KFold++.xlsx")
model_save_dir = os.path.join(logs_path, "models")

# Params
image_size = 180
batch_size = 32
epochs = 40
num_classes = 5
n_splits = 5

# CLAHE Preprocessing
def clahe_preprocess(img_path):
    img = cv2.imread(img_path.decode(), cv2.IMREAD_COLOR)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(2.0, (8, 8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    final = cv2.cvtColor(merged, cv2.COLOR_LAB2RGB)
    final = cv2.resize(final, (image_size, image_size))
    return final.astype(np.uint8)

def load_dataset(paths, labels):
    def _process(path, label):
        image = tf.numpy_function(clahe_preprocess, [path], tf.uint8)
        image.set_shape([image_size, image_size, 3])
        image = tf.image.convert_image_dtype(image, tf.float32)
        return image, tf.one_hot(label, num_classes)
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(_process, num_parallel_calls=tf.data.AUTOTUNE)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Build Model
def build_model():
    model = Sequential([
        Conv2D(32, (3,3), padding='same', input_shape=(image_size, image_size, 3)),
        BatchNormalization(),
        LeakyReLU(),
        MaxPooling2D(),

        Conv2D(64, (3,3), padding='same'),
        BatchNormalization(),
        LeakyReLU(),
        MaxPooling2D(),

        Conv2D(128, (3,3), padding='same'),
        BatchNormalization(),
        LeakyReLU(),
        MaxPooling2D(),

        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(128),
        LeakyReLU(),
        Dropout(0.4),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Cosine Annealing
def cosine_annealing(epoch):
    lr_max, lr_min = 3e-4, 1e-6
    return lr_min + 0.5 * (lr_max - lr_min) * (1 + np.cos(np.pi * epoch / epochs))

# Load Image Paths
data = []
class_names = sorted([d for d in os.listdir(dataset_path) if not d.startswith('.')])
class_to_idx = {cls: i for i, cls in enumerate(class_names)}

for cls in class_names:
    folder = os.path.join(dataset_path, cls)
    for fname in os.listdir(folder):
        if fname.lower().endswith(('jpg', 'jpeg', 'png')):
            data.append((os.path.join(folder, fname), class_to_idx[cls]))

data_df = pd.DataFrame(data, columns=["filepath", "label"])
x, y = data_df.filepath.values, data_df.label.values

# K-Fold Training
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(x, y), 1):
    print(f"\n===== Fold {fold} =====")
    x_train, y_train = x[train_idx], y[train_idx]
    x_val, y_val = x[val_idx], y[val_idx]

    train_data = load_dataset(x_train, y_train)
    val_data = load_dataset(x_val, y_val)

    model = build_model()
    model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = {i: w for i, w in enumerate(class_weights)}

    callbacks = [
        ModelCheckpoint(os.path.join(model_save_dir, f"customCNNpp_fold{fold}.keras"), monitor='val_loss', save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5),
        CSVLogger(os.path.join(logs_path, f"results/customCNN_OCTID_fold{fold}.csv")),
        LearningRateScheduler(cosine_annealing)
    ]

    model.fit(train_data, validation_data=val_data, epochs=epochs, callbacks=callbacks, class_weight=class_weight_dict)

    # Evaluation
    scores = model.evaluate(val_data, verbose=0)
    y_pred = np.argmax(model.predict(val_data), axis=1)
    y_true = np.argmax(np.concatenate([y.numpy() for _, y in val_data], axis=0), axis=1)
    report = classification_report(y_true, y_pred, output_dict=True)

    fold_results.append({
        "Fold": fold,
        "Test Accuracy": scores[1],
        "Test Loss": scores[0],
        "Precision": report['weighted avg']['precision'],
        "Recall": report['weighted avg']['recall'],
        "F1-Score": report['weighted avg']['f1-score']
    })

# Save Final Results
final_df = pd.DataFrame(fold_results)
final_df.to_excel(result_path, index=False)
print("✅ All folds complete. Results saved.")



===== Fold 1 =====
Epoch 1/40


/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 367ms/step - accuracy: 0.1877 - loss: 2.6497 - val_accuracy: 0.1724 - val_loss: 1.6086 - learning_rate: 3.0000e-04
Epoch 2/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 203ms/step - accuracy: 0.2222 - loss: 2.3221 - val_accuracy: 0.1724 - val_loss: 1.5973 - learning_rate: 2.9954e-04
Epoch 3/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 206ms/step - accuracy: 0.2328 - loss: 2.2702 - val_accuracy: 0.1724 - val_loss: 1.5881 - learning_rate: 2.9816e-04
Epoch 4/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step - accuracy: 0.2409 - loss: 2.3103 - val_accuracy: 0.1724 - val_loss: 1.5812 - learning_rate: 2.9587e-04
Epoch 5/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 209ms/step - accuracy: 0.2378 - loss: 2.4041 - val_accuracy: 0.1724 - val_loss: 1.5806 - learning_rate: 2.9268e-04
Epoch 6/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 210ms/step - accuracy: 0.2266 - loss: 2.3452 - val_accuracy: 0.1724 - val_loss: 1.5865 - learning_rate: 2.8862e-04
Epoch 7/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 197ms/step - accuracy: 0.2052 - loss: 2.4767 - v

2025-04-07 19:46:53.431545: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is il

8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 278ms/step - accuracy: 0.1091 - loss: 3.1747 - val_accuracy: 0.3621 - val_loss: 1.5930 - learning_rate: 3.0000e-04
Epoch 2/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 222ms/step - accuracy: 0.1973 - loss: 2.2758 - val_accuracy: 0.3621 - val_loss: 1.5973 - learning_rate: 2.9954e-04
Epoch 3/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step - accuracy: 0.2071 - loss: 2.1285 - val_accuracy: 0.1724 - val_loss: 1.5998 - learning_rate: 2.9816e-04
Epoch 4/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 201ms/step - accuracy: 0.1625 - loss: 2.2810 - val_accuracy: 0.2241 - val_loss: 1.6027 - learning_rate: 1.4793e-04
Epoch 5/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step - accuracy: 0.2107 - loss: 2.3438 - val_accuracy: 0.1724 - val_loss: 1.6085 - learning_rate: 2.9268e-04
Epoch 6/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 181ms/step - accuracy: 0.1495 - loss: 2.2849 - val_accuracy: 0.1724 - val_loss: 1.6143 - learning_rate: 2.8862e-04
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step

===== Fold 3 =====
Epoch 1/40


...

2025-04-07 19:47:23.957393: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is il

8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 319ms/step - accuracy: 0.1470 - loss: 2.9082 - val_accuracy: 0.1930 - val_loss: 1.6079 - learning_rate: 3.0000e-04
Epoch 2/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 240ms/step - accuracy: 0.1848 - loss: 2.2997 - val_accuracy: 0.1930 - val_loss: 1.6018 - learning_rate: 2.9954e-04
Epoch 3/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 223ms/step - accuracy: 0.2115 - loss: 2.3454 - val_accuracy: 0.1930 - val_loss: 1.5964 - learning_rate: 2.9816e-04
Epoch 4/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 940ms/step - accuracy: 0.1743 - loss: 2.2858 - val_accuracy: 0.1404 - val_loss: 1.5928 - learning_rate: 2.9587e-04
Epoch 5/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 215ms/step - accuracy: 0.1583 - loss: 2.3307 - val_accuracy: 0.2105 - val_loss: 1.5926 - learning_rate: 2.9268e-04
Epoch 6/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 180ms/step - accuracy: 0.1872 - loss: 2.4721 - val_accuracy: 0.1930 - val_loss: 1.5934 - learning_rate: 2.8862e-04
Epoch 7/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 213ms/step - accuracy: 0.2177 - loss: 2.1940 - v

2025-04-07 19:47:50.812368: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is il

8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 295ms/step - accuracy: 0.1376 - loss: 3.7488 - val_accuracy: 0.1754 - val_loss: 1.6072 - learning_rate: 3.0000e-04
Epoch 2/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - accuracy: 0.1887 - loss: 2.6862 - val_accuracy: 0.1754 - val_loss: 1.6125 - learning_rate: 2.9954e-04
Epoch 3/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 283ms/step - accuracy: 0.2126 - loss: 2.5219 - val_accuracy: 0.1754 - val_loss: 1.6210 - learning_rate: 2.9816e-04
Epoch 4/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 228ms/step - accuracy: 0.1892 - loss: 2.4374 - val_accuracy: 0.1754 - val_loss: 1.6305 - learning_rate: 1.4793e-04
Epoch 5/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 215ms/step - accuracy: 0.1591 - loss: 2.4095 - val_accuracy: 0.1754 - val_loss: 1.6444 - learning_rate: 2.9268e-04
Epoch 6/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 221ms/step - accuracy: 0.2187 - loss: 2.2148 - val_accuracy: 0.1754 - val_loss: 1.6645 - learning_rate: 2.8862e-04
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
✅ All folds complete. Results saved.


2025-04-07 19:48:05.861233: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mananmathur/anaconda3/envs/oct_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is il